In [2]:
pip install geojson

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%conda install folium

Channels:
 - defaults
Platform: osx-64
doneecting package metadata (repodata.json): - 
- lving environment: | 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from bs4 import BeautifulSoup
import time # For adding a delay between requests
import random # For randomizing delay
import requests

In [ ]:
nash_list = pd.read_csv('../data/listings.csv')

In [ ]:
nash_list.info()

In [ ]:
nash_list[['latitude','longitude']].head(50)

In [ ]:
nash_list.loc[nash_list['minimum_minimum_nights']>=30]

In [ ]:
nash_review = pd.read_csv('../data/reviews.csv')

In [ ]:
minof30ni = nash_list.loc[nash_list['minimum_nights'] >= 30]
minof30ni

In [ ]:
len(set(nash_review))

In [ ]:
minof30ni['minimum_nights'].min()

In [ ]:
maxminof30ni =minof30ni.loc[minof30ni['minimum_nights'] == 730]
maxminof30ni['price']

In [ ]:
nash_neighbors = pd.read_csv('../data/neighbourhoods.csv')
nash_neighbors

In [ ]:
##parksgpd['lng'] = parksgpd.geometry.centroid.x
##parksgpd['lat']= parksgpd.geometry.centroid.y
##parksgpd

In [ ]:
zillow_list = pd.read_csv('../data/zillow.csv')

In [ ]:
zillow_list

In [ ]:
nash_zillow = zillow_list.loc[zillow_list['City']== 'Nashville']
nash_zillow

In [ ]:
zillow_list.info(verbose=True, show_counts=True)

In [ ]:
incidents = pd.read_csv('../data/incidents.csv')

In [ ]:
latnlongairbnb = pd.read_csv('../data/latnlongairbnb.csv')
latnlongairbnb

In [ ]:
# Set up geolocator
##geolocator = Nominatim(user_agent="zip_code_lookup")
##reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Function to get ZIP code from coordinates
##def get_zipcode(lat, lon):
  #  try:
   #     location = reverse((lat, lon), exactly_one=True)
    #    if location and 'postcode' in location.raw['address']:
     #       return location.raw['address']['postcode']
    #except:
     #   return None
    #return None

# Apply to dataframe
#minof30ni['zipcode'] = minof30ni.apply(
 #   lambda row: get_zipcode(row['latitude'], row['longitude']) if pd.notnull(row['latitude']) and pd.notnull(row['longitude']) else None,
  #  axis=1
#)

# Save updated file
#output_file = "minof30ni_with_zip.csv"
#minof30ni.to_csv(output_file, index=False)
#print(f"ZIP codes added. File saved as {output_file}")

In [ ]:
min30ni_zip = pd.read_csv('../data/minof30ni_with_zip.csv')

In [ ]:
min30ni_zip

In [ ]:
incidents.info()

In [ ]:
incidents['ZIP_Code'].unique()

In [ ]:
min30ni_zip['zipcode'].unique()

In [ ]:
nash_zillow['RegionName'].unique()

In [ ]:
nash_min30 = min30ni_zip[min30ni_zip['zipcode'].isin(nash_zillow['RegionName'])]
nash_min30['zipcode'].unique()

In [ ]:
min30ni_zip

In [ ]:
minof30ni_zip = minof30ni_zip.drop(columns='price')

In [ ]:
price_for_rent = pd.read_csv('../data/price_per30ni.csv')

In [ ]:
price_for_rent

In [ ]:
price_min_30ni = pd.merge(nash_min30, price_for_rent, on=['id'])

In [ ]:
price_min_30ni